In [1]:
!pip install numpy==1.25.0
!pip install torch==2.6.0
!pip install torchaudio
!pip install lightning

In [2]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install text-unidecode

# ## Install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

## Install TorchAudio
!pip install torchaudio>=0.10.0 -f https://download.pytorch.org/whl/torch_stable.html

## Grab the config we'll use in this example
!mkdir configs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsndfile1 is already the newest version (1.0.31-2ubuntu0.2).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
sox is already the newest version (14.4.2+git20190427-2+deb11u2ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
DEPRECATION: git+https://github.com/NVIDIA/NeMo.git@main#egg=nemo_toolkit[all] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Cloning https://github.com/NVIDIA/NeMo.git (to revision main) to /tmp/pip-install-wxgq6_39/nemo-toolkit_bc08a1c0cbab42c6bcdb8feaa4d73178
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/NeMo.git /tmp/pip-install-wxgq6_39/nemo-toolkit_bc08a1c0cbab42c6bcdb8feaa4d73178
  Re

# minGPT License

*This notebook port's the [minGPT codebase](https://github.com/karpathy/minGPT) into equivalent NeMo code. The license for minGPT has therefore been attached here.*

```
The MIT License (MIT) Copyright (c) 2020 Andrej Karpathy

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
```

# torch-rnn License
*This notebook utilizes the `tiny-shakespeare` dataset from the [torch-rnn](https://github.com/jcjohnson/torch-rnn) codebase. The license for torch-rnn has therefore been attached here.*

```
The MIT License (MIT)

Copyright (c) 2016 Justin Johnson

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
```


-------

***Note: This notebook will intentionally introduce some errors to show the power of Neural Types or model development concepts, inside the cells marked with `[ERROR CELL]`. The explanation of and resolution of such errors can be found in the subsequent cells.***

-----

# The NeMo Model

NeMo comes with several state-of-the-art pre-trained Conversational AI models for users to quickly be able to start training and fine-tuning on their own datasets.   

In the previous [NeMo Primer](https://colab.research.google.com/github/NVIDIA/NeMo/blob/stable/tutorials/00_NeMo_Primer.ipynb) notebook, we learned how to download pretrained checkpoints with NeMo and we also discussed the fundamental concepts of the NeMo Model. The previous tutorial showed us how to use, modify, save, and restore NeMo Models.

In this tutorial we will learn how to develop a non-trivial NeMo model from scratch. This helps us to understand the underlying components and how they interact with the overall PyTorch ecosystem.


-------
At the heart of NeMo lies the concept of the "Model". For NeMo developers, a "Model" is the neural network(s) as well as all the infrastructure supporting those network(s), wrapped into a singular, cohesive unit. As such, most NeMo models are constructed to contain the following out of the box (note: some NeMo models support additional functionality specific to the domain/use case!) -

 -  Neural Network architecture - all of the modules that are required for the model.

 -  Dataset + Data Loaders - all of the components that prepare the data for consumption during training or evaluation.

 -  Preprocessing + Postprocessing - any of the components that process the datasets so the modules can easily consume them.

 -  Optimizer + Schedulers - basic defaults that work out of the box and allow further experimentation with ease.

 - Any other supporting infrastructure - tokenizers, language model configuration, data augmentation, etc.

# Constructing a NeMo Model

NeMo "Models" are comprised of a few key components, so let's tackle them one by one. We will attempt to go in the order that's stated above.

To make this slightly challenging, let's port a model from the NLP domain this time. Transformers are all the rage, with BERT and his friends from Sesame Street forming the core infrastructure for many NLP tasks.

An excellent (yet simple) implementation of one such model - GPT - can be found in the `minGPT` repository - https://github.com/karpathy/minGPT. While the script is short, it explains and succinctly explores all of the core components we expect in a NeMo model, so it's a prime candidate for NeMo! Sidenote: NeMo supports GPT in its NLP collection, and as such, this notebook aims to be an in-depth development walkthrough for such models.

In the following notebook, we will attempt to port minGPT to NeMo, and along the way, discuss some core concepts of NeMo itself.

# Constructing the Neural Network Architecture

First, on the list - the neural network that forms the backbone of the NeMo Model.

So how do we create such a model? Using PyTorch! As you'll see below, NeMo components are compatible with all of PyTorch, so you can augment your workflow without ever losing the flexibility of PyTorch itself!

Let's start with a couple of imports -

In [3]:
import torch
import nemo
from nemo.core import NeuralModule
from nemo.core import typecheck

## Neural Module
Wait, what's `NeuralModule`? Where is the wonderful `torch.nn.Module`?

`NeuralModule` is a subclass of `torch.nn.Module`, and it brings with it a few additional functionalities.

In addition to being a `torch.nn.Module`, thereby being entirely compatible with the PyTorch ecosystem, it has the following capabilities -

1) `Typing` - It adds support for `Neural Type Checking` to the model. `Typing` is optional but quite useful, as we will discuss below!

2) `Serialization` - Remember the `OmegaConf` config dict and YAML config files? Well, all `NeuralModules` inherently supports serialization/deserialization from such config dictionaries!

3) `FileIO` - This is another entirely optional file serialization system. Does your `NeuralModule` require some way to preserve data that can't be saved into a PyTorch checkpoint? Write your serialization and deserialization logic in two handy methods! **Note**: When you create the final NeMo Model, this will be implemented for you! Automatic serialization and deserialization support of NeMo models!


In [5]:
class MyEmptyModule(NeuralModule):

  def forward(self):
    print("Neural Module ~ hello world!")

In [6]:
x = MyEmptyModule()
x()

Neural Module ~ hello world!


## Neural Types

Neural Types? You might be wondering what that term refers to.

Almost all NeMo components inherit the class `Typing`. `Typing` is a simple class that adds two properties to the class that inherits it - `input_types` and `output_types`. A NeuralType, by its shortest definition, is simply a semantic tensor. It contains information regarding the semantic shape the tensor should hold, as well as the semantic information of what that tensor represents. That's it.

So what semantic information does such a typed tensor contain? Let's take an example below.




------
Across the Deep Learning domain, we often encounter cases where tensor shapes may match, but the semantics don't match at all. For example take a look at the following rank 3 tensors -

In [7]:
# Case 1:
embedding = torch.nn.Embedding(num_embeddings=10, embedding_dim=30)
x = torch.randint(high=10, size=(1, 5))
print("x :", x)
print("embedding(x) :", embedding(x).shape)

x : tensor([[6, 1, 0, 8, 2]])
embedding(x) : torch.Size([1, 5, 30])


In [8]:
# Case 2
lstm = torch.nn.LSTM(1, 30, batch_first=True)
x = torch.randn(1, 5, 1)
print("x :", x)
print("lstm(x) :", lstm(x)[0].shape)  # Let's take all timestep outputs of the LSTM

x : tensor([[[-0.9169],
         [ 0.0443],
         [-1.0155],
         [ 1.2787],
         [-0.8890]]])
lstm(x) : torch.Size([1, 5, 30])


-------
As you can see, the output of Case 1 is an embedding of shape [1, 5, 30], and the output of Case 2 is an LSTM output (state `h` over all time steps), also of the same shape [1, 5, 30].

Do they have the same shape? **Yes**. <br>If we do a Case 1 .shape == Case 2 .shape, will we get True as an output? **Yes**. <br>
Do they represent the same concept? **No**. <br>


The ability to recognize that the two tensors do not represent the same semantic information is precisely why we utilize Neural Types. It contains the information of both the shape and the semantic concept of what that tensor represents. If we performed a neural type check between the two outputs of those tensors, it would raise an error saying semantically they were different things (more technically, it would say that they are `INCOMPATIBLE` with each other)!


--------

You may have read of concepts such as [Named Tensors](https://pytorch.org/docs/stable/named_tensor.html). While conceptually similar, Neural Types attached by NeMo are not as tightly bound to the PyTorch ecosystem - practically any object of a class can be attached with a neural type!


## Neural Types - Usage

Neural Types sound interesting, so how do we go about adding them? Let's take a few cases below.

Neural Types are one of the core foundations of NeMo - you will find them in a vast majority of Neural Modules, and every NeMo Model will have its Neural Types defined. While they are entirely optional and not intrusive, NeMo takes great care to support it so that there is no semantic incompatibility between components being used by users.

Let's start with a basic example of a type checked module.

In [9]:
from nemo.core.neural_types import NeuralType
from nemo.core.neural_types import *

In [10]:
class EmbeddingModule(NeuralModule):
  def __init__(self):
    super().__init__()
    self.embedding = torch.nn.Embedding(num_embeddings=10, embedding_dim=30)

  @typecheck()
  def forward(self, x):
    return self.embedding(x)

  @property
  def input_types(self):
    return {
        'x': NeuralType(axes=('B', 'T'), elements_type=Index())
    }

  @property
  def output_types(self):
    return {
        'y': NeuralType(axes=('B', 'T', 'C'), elements_type=EmbeddedTextType())
    }

To show the benefit of Neural Types, we are going to replicate the above cases inside NeuralModules.

Let's discuss how we added type checking support to the above class.

1) `forward` has a decorator `@typecheck()` on it.

2) `input_types` and `output_types` properties are defined.

That's it!

-------

Let's expand on each of the above steps.

- `@typecheck()` is a simple decorator that takes any class that inherits `Typing` (NeuralModule does this for us) and adds the two default properties of `input_types` and `output_types`, which by default returns None.

The `@typecheck()` decorator's explicit use ensures that, by default, neural type checking is **disabled**. NeMo does not wish to intrude on the development process of models. So users can "opt-in" to type checking by overriding the two properties. Therefore, the decorator ensures that users are not burdened with type checking before they wish to have it.

So what is `@typecheck()`? Simply put, you can wrap **any** function of a class that inherits `Typing` with this decorator, and it will look up the definition of the types of that class and enforce them. Typically, `torch.nn.Module` subclasses only implement `forward()` so it is most common to wrap that method, but `@typecheck()` is a very flexible decorator. Inside NeMo, we will show some advanced use cases (which are quite crucial to particular domains such as TTS).

------

As we see above, `@typecheck()` enforces the types. How then, do we provide this type of information to NeMo?

By overriding `input_types` and `output_types` properties of the class, we can return a dictionary mapping a string name to a `NeuralType`.

In the above case, we define a `NeuralType` as two components -

- `axes`: This is the semantic information of the carried by the axes themselves. The most common axes information is from single character notation.

> `B` = Batch <br>
> `C` / `D` - Channel / Dimension (treated the same) <br>
> `T` - Time <br>
> `H` / `W` - Height / Width <br>

- `elements_type`: This is the semantic information of "what the tensor represents". All such types are derived from the basic `ElementType`, and merely subclassing `ElementType` allows us to build a hierarchy of custom semantic types that can be used by NeMo!

Here, we declare that the input is an element_type of `Index` (index of the character in the vocabulary) and that the output is an element_type of `EmbeddedTextType` (the text embedding)

In [11]:
embedding_module = EmbeddingModule()

Now let's construct the equivalent of the Case 2 above, but as a `NeuralModule`.

In [12]:
class LSTMModule(NeuralModule):
  def __init__(self):
    super().__init__()
    self.lstm = torch.nn.LSTM(1, 30, batch_first=True)

  @typecheck()
  def forward(self, x):
    return self.lstm(x)

  @property
  def input_types(self):
    return {
        'x': NeuralType(axes=('B', 'T', 'C'), elements_type=SpectrogramType())
    }

  @property
  def output_types(self):
    return {
        'y': NeuralType(axes=('B', 'T', 'C'), elements_type=EncodedRepresentation())
    }

------
Here, we define the LSTM module from the Case 2 above.

We changed the input to be a rank three tensor, now representing a "SpectrogramType". We intentionally keep it generic - it can be a `MelSpectrogramType` or a `MFCCSpectrogramType` as its input!

The output of an LSTM is now an `EncodedRepresentation`. Practically, this can be the output of a CNN layer, a Transformer block, or in this case, an LSTM layer. We can, of course, specialize by subclassing EncodedRepresentation and then using that!

In [13]:
lstm_module = LSTMModule()

------
Now for the test !

In [14]:
# Case 1 [ERROR CELL]
x1 = torch.randint(high=10, size=(1, 5))
print("x :", x1)
print("embedding(x) :", embedding_module(x1).shape)

x : tensor([[8, 7, 9, 9, 7]])


TypeError: All arguments must be passed by kwargs only for typed methods

-----
You might be wondering why we get a `TypeError` right off the bat. This `TypeError` is raised by design.

Positional arguments can cause significant issues during model development, mostly when the model/module design is not finalized. To reduce the potential for mistakes caused by wrong positional arguments and enforce the name of arguments provided to the function, `Typing` requires you to **call all of your type-checked functions by kwargs only**.

In [15]:
# Case 1
print("x :", x1)
print("embedding(x) :", embedding_module(x=x1).shape)

x : tensor([[8, 7, 9, 9, 7]])
embedding(x) : torch.Size([1, 5, 30])


Now let's try the same for the `LSTMModule` in Case 2

In [16]:
# Case 2 [ERROR CELL]
x2 = torch.randn(1, 5, 1)  # Input = [B=1, T=5, C=1]
print("x :", x2)
print("lstm(x) :", lstm_module(x=x2)[0].shape)  # Let's take all timestep outputs of the LSTM

x : tensor([[[-0.2373],
         [ 0.1684],
         [ 0.5471],
         [ 1.3196],
         [-0.0680]]])


TypeError: Number of output arguments provided (2) is not as expected. It should be larger or equal than 1 and less or equal than 1.
This can be either because insufficient/extra number of output NeuralTypes were provided,or the provided NeuralTypes {'y': NeuralType(axis=(batch, time, dimension), element_type=EncodedRepresentation)} should enable container support (add '[]' to the NeuralType definition)

-----
Now we get a type error stating that the number of output arguments provided does not match what is expected.

What exactly is going on here? Well, inside our `LSTMModule` class, we declare the output types to be a single NeuralType - an `EncodedRepresentation` of shape [B, T, C].

But the output of an LSTM layer is a tuple of
1) the encoded representation of shape [B, T, C]
2) another tuple containing two state values - the hidden state `h` and the cell state `c`, each of shape [num_layers * num_directions, B, C]!

So the neural type system raises an error saying that the number of output arguments does not match what is expected.

**NOTE**: The axis kind information of the two states will be represented by `D` to represent a general "Dimension" - since `num_layers` and `num_directions` are collapsed under a single axis. For NeMo, Axis types of `C` and `D` are equivalent and can be interchanged, so we will use `C` here to represent the hidden dimension of the LSTM and `D` to represent the merged axis `num_layers * num_directions`.

Let's fix the above.

In [17]:
class CorrectLSTMModule(LSTMModule):  # Let's inherit the wrong class to make it easy to override
  @property
  def output_types(self):
    return {
        'y': NeuralType(axes=('B', 'T', 'C'), elements_type=EncodedRepresentation()),
        'h_c': [NeuralType(axes=('D', 'B', 'C'), elements_type=EncodedRepresentation())],
    }

You should note that for the `h_c` neural type, we wrap it in a list - `[]`. NeMo, by default, assumes that each `NeuralType` corresponds to a single returned value. However, in the case of LSTMs, they produce a tuple of two state tensors.

So we inform NeMo that this particular `NeuralType` is a single-dimensional list of items - and that each element of this list shares the same `NeuralType` and has the same shape.

NeMo then ensures that the `h_c` is always a list of tensors. It will not check *how many* items are in the list, but will ensure that the returned value *must be a list containing zero or more items* - and that each of these items share the same `NeuralType`.

In [18]:
lstm_module = CorrectLSTMModule()

In [19]:
# Case 2
x2 = torch.randn(1, 5, 1)
y2, (h, c) = lstm_module(x=x2)
print("x :", x2)
print("lstm(x) :", y2.shape)  # The output of the LSTM RNN
print("hidden state (h) :", h.shape)  # The first hidden state of the LSTM RNN
print("hidden state (c) :", c.shape)  # The second hidden state of the LSTM RNN

x : tensor([[[-0.0403],
         [ 1.6089],
         [-1.1197],
         [-0.2711],
         [-0.0860]]])
lstm(x) : torch.Size([1, 5, 30])
hidden state (h) : torch.Size([1, 1, 30])
hidden state (c) : torch.Size([1, 1, 30])


------
Great! So now, the type checking system is happy.

If you looked closely, the outputs were ordinary Torch Tensors (this is good news; we don't want to be incompatible with torch Tensors after all!). So, where exactly is the type of information stored?

When the `output_types` is overridden, and valid torch tensors are returned as a result, these tensors are attached with the attribute `neural_type`. Let's inspect this -

In [20]:
emb_out = embedding_module(x=x1)
lstm_out = lstm_module(x=x2)[0]

assert hasattr(emb_out, 'neural_type')
assert hasattr(lstm_out, 'neural_type')

In [21]:
print("Embedding tensor :", emb_out.neural_type)
print("LSTM tensor :", lstm_out.neural_type)

Embedding tensor : axes: (batch, time, dimension); elements_type: EmbeddedTextType
LSTM tensor : axes: (batch, time, dimension); elements_type: EncodedRepresentation


-------
So we see that these tensors now have this attribute called `neural_type` and are the same shape.

This exercise's entire goal was to assert that the two outputs are semantically **not** the same object, even if they are the same shape.

Let's test this!

In [22]:
emb_out.neural_type.compare(lstm_out.neural_type)

<NeuralTypeComparisonResult.INCOMPATIBLE: 6>

In [23]:
emb_out.neural_type == lstm_out.neural_type

<NeuralTypeComparisonResult.INCOMPATIBLE: 6>

## Neural Types - Limitations

You might have noticed one interesting fact - our inputs were just `torch.Tensor` to both typed function calls, and they had no `neural_type` assigned to them.

So why did the type check system not raise any error?

This is to maintain compatibility - type checking is meant to work on a chain of function calls - and each of these functions should themselves be wrapped with the `@typecheck()` decorator. This is also done because we don't want to overtax the forward call with dozens of checks, and therefore we only type modules that perform some higher-order logical computation.

------

As an example, it is mostly unnecessary (but still possible) to type the input and output of every residual block of a ResNet model. However, it is practically important to type the encoder (no matter how many layers is inside it) and the decoder (the classification head) separately so that when one does fine-tuning, there is no semantic mismatch of the tensors input to the encoder and bound to the decoder.

-------
For this case, since it would be impractical to extend a class to attach a type to the input tensor, we can take a shortcut and directly attach the neural type to the input!

In [24]:
embedding_module = EmbeddingModule()
x1 = torch.randint(high=10, size=(1, 5))

# Attach correct neural type
x1.neural_type = NeuralType(('B', 'T'), Index())

print("embedding(x) :", embedding_module(x=x1).shape)

embedding(x) : torch.Size([1, 5, 30])


In [25]:
# Attach wrong neural type [ERROR CELL]
x1.neural_type = NeuralType(('B', 'T'), LabelsType())

print("embedding(x) :", embedding_module(x=x1).shape)

TypeError: NeuralTypeComparisonResult.INCOMPATIBLE :
Input type expected : axes: (batch, time); elements_type: Index
Input type found : axes: (batch, time); elements_type: LabelsType
Argument: x

## Let's create the minGPT components

Now that we have a somewhat firm grasp of neural type checking, let's begin porting the minGPT example code. Once again, most of the code will be a direct port from the [minGPT repository](https://github.com/karpathy/minGPT).

Here, you will notice one thing. By just changing class imports, one `@typecheck()` on forward, and adding `input_types` and `output_types` (which are also entirely optional!), we are almost entirely done with the PyTorch Lightning port!

In [26]:
import math
from typing import List, Set, Dict, Tuple, Optional

import torch
import torch.nn as nn
from torch.nn import functional as F

## Creating Element Types

Till now, we have used the Neural Types provided by the NeMo core. But we need not be restricted to the pre-defined element types !

Users have total flexibility in defining any hierarchy of element types as they please!

In [27]:
class AttentionType(EncodedRepresentation):
  """Basic Attention Element Type"""

class SelfAttentionType(AttentionType):
  """Self Attention Element Type"""

class CausalSelfAttentionType(SelfAttentionType):
  """Causal Self Attention Element Type"""

## Creating the modules

Neural Modules are generally top-level modules but can be used at any level of the module hierarchy.

For demonstration, we will treat an encoder comprising a block of Causal Self Attention modules as a typed Neural Module. Of course, we can also treat each Causal Self Attention layer itself as a neural module if we require it, but top-level modules are generally preferred.

In [28]:
class CausalSelfAttention(nn.Module):
    """
    A vanilla multi-head masked self-attention layer with a projection at the end.
    It is possible to use torch.nn.MultiheadAttention here but I am including an
    explicit implementation here to show that there is nothing too scary here.
    """

    def __init__(self, n_embd, block_size, n_head, attn_pdrop, resid_pdrop):
        super().__init__()
        assert n_embd % n_head == 0
        self.n_head = n_head
        # key, query, value projections for all heads
        self.key = nn.Linear(n_embd, n_embd)
        self.query = nn.Linear(n_embd, n_embd)
        self.value = nn.Linear(n_embd, n_embd)
        # regularization
        self.attn_drop = nn.Dropout(attn_pdrop)
        self.resid_drop = nn.Dropout(resid_pdrop)
        # output projection
        self.proj = nn.Linear(n_embd, n_embd)
        # causal mask to ensure that attention is only applied to the left in the input sequence
        self.register_buffer("mask", torch.tril(torch.ones(block_size, block_size))
                                     .view(1, 1, block_size, block_size))
    def forward(self, x, layer_past=None):
        B, T, C = x.size()

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        k = self.key(x).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = self.query(x).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = self.value(x).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(self.mask[:,:,:T,:T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        att = self.attn_drop(att)
        y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_drop(self.proj(y))
        return y


class Block(nn.Module):
    """ an unassuming Transformer block """

    def __init__(self, n_embd, block_size, n_head, attn_pdrop, resid_pdrop):
        super().__init__()
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        self.attn = CausalSelfAttention(n_embd, block_size, n_head, attn_pdrop, resid_pdrop)
        self.mlp = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.GELU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(resid_pdrop),
        )

    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x

## Building the NeMo Model

Since a NeMo Model is comprised of various parts, we are going to iterate on the model step by step inside this notebook. As such, we will have multiple intermediate NeMo "Models", which will be partial implementations, and they will inherit each other iteratively.

In a complete implementation of a NeMo Model (as found in the NeMo collections), all of these components will generally be found in a single class.

Let's start by inheriting `ModelPT` - the core class of a PyTorch NeMo Model, which inherits the PyTorch Lightning Module.

-------
**Remember**:

 - The NeMo equivalent of `torch.nn.Module` is the `NeuralModule.
 - The NeMo equivalent of the `LightningModule` is `ModelPT`.


In [29]:
import lightning.pytorch as ptl
from nemo.core import ModelPT
from omegaconf import OmegaConf

------
Next, let's construct the bare minimum implementation of the NeMo Model - just the constructor, the initializer of weights, and the forward method.

Initially, we will follow the steps followed by the minGPT implementation, and progressively refactor for NeMo

In [30]:
class PTLGPT(ptl.LightningModule):
  def __init__(self,
                 # model definition args
                 vocab_size: int, # size of the vocabulary (number of possible tokens)
                 block_size: int, # length of the model's context window in time
                 n_layer: int, # depth of the model; number of Transformer blocks in sequence
                 n_embd: int, # the "width" of the model, number of channels in each Transformer
                 n_head: int, # number of heads in each multi-head attention inside each Transformer block
                 # model optimization args
                 learning_rate: float = 3e-4, # the base learning rate of the model
                 weight_decay: float = 0.1, # amount of regularizing L2 weight decay on MatMul ops
                 betas: Tuple[float, float] = (0.9, 0.95), # momentum terms (betas) for the Adam optimizer
                 embd_pdrop: float = 0.1, # \in [0,1]: amount of dropout on input embeddings
                 resid_pdrop: float = 0.1, # \in [0,1]: amount of dropout in each residual connection
                 attn_pdrop: float = 0.1, # \in [0,1]: amount of dropout on the attention matrix
                 ):
        super().__init__()

        # save these for optimizer init later
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.betas = betas

        # input embedding stem: drop(content + position)
        self.tok_emb = nn.Embedding(vocab_size, n_embd)
        self.pos_emb = nn.Parameter(torch.zeros(1, block_size, n_embd))
        self.drop = nn.Dropout(embd_pdrop)
        # deep transformer: just a sequence of transformer blocks
        self.blocks = nn.Sequential(*[Block(n_embd, block_size, n_head, attn_pdrop, resid_pdrop) for _ in range(n_layer)])
        # decoder: at the end one more layernorm and decode the answers
        self.ln_f = nn.LayerNorm(n_embd)
        self.head = nn.Linear(n_embd, vocab_size, bias=False) # no need for extra bias due to one in ln_f

        self.block_size = block_size
        self.apply(self._init_weights)

        print("number of parameters: %e" % sum(p.numel() for p in self.parameters()))

  def forward(self, idx):
      b, t = idx.size()
      assert t <= self.block_size, "Cannot forward, model block size is exhausted."

      # forward the GPT model
      token_embeddings = self.tok_emb(idx) # each index maps to a (learnable) vector
      position_embeddings = self.pos_emb[:, :t, :] # each position maps to a (learnable) vector
      x = self.drop(token_embeddings + position_embeddings)
      x = self.blocks(x)
      x = self.ln_f(x)
      logits = self.head(x)

      return logits

  def get_block_size(self):
      return self.block_size

  def _init_weights(self, module):
      """
      Vanilla model initialization:
      - all MatMul weights \in N(0, 0.02) and biases to zero
      - all LayerNorm post-normalization scaling set to identity, so weight=1, bias=0
      """
      if isinstance(module, (nn.Linear, nn.Embedding)):
          module.weight.data.normal_(mean=0.0, std=0.02)
          if isinstance(module, nn.Linear) and module.bias is not None:
              module.bias.data.zero_()
      elif isinstance(module, nn.LayerNorm):
          module.bias.data.zero_()
          module.weight.data.fill_(1.0)

------
Let's create a PyTorch Lightning Model above, just to make sure it works !

In [31]:
m = PTLGPT(vocab_size=100, block_size=32, n_layer=1, n_embd=32, n_head=4)

number of parameters: 2.019200e+04


------
Now, let's convert the above easily into a NeMo Model.

A NeMo Model constructor generally accepts only two things -

1) `cfg`: An OmegaConf DictConfig object that defines precisely the components required by the model to define its neural network architecture, data loader setup, optimizer setup, and any additional components needed for the model itself.

2) `trainer`: An optional Trainer from PyTorch Lightning if the NeMo model will be used for training. It can be set after construction (if required) using the `set_trainer` method. For this notebook, we will not be constructing the config for the Trainer object.

## Refactoring Neural Modules

As we discussed above, Neural Modules are generally higher-level components of the Model and can potentially be replaced by equivalent Neural Modules.

As we see above, the embedding modules, deep transformer decoder network, and final decoder layer have all been combined inside the PyTorch Lightning implementation constructor.

------

However, the final decoder module could have been an RNN instead of a simple Linear layer, or it could have been a 1D-CNN instead.

Likewise, the deep transformer decoder could potentially have a different implementation of Self Attention modules.

These changes cannot be easily implemented any more inside the above implementation. However, if we refactor these components into their respective NeuralModules, then we can easily replace them with equivalent modules we construct in the future!

### Refactoring the Embedding module

Let's first refactor out the embedding module from the above implementation

In [32]:
class GPTEmbedding(NeuralModule):
  def __init__(self, vocab_size: int, n_embd: int, block_size: int, embd_pdrop: float = 0.0):
    super().__init__()

    # input embedding stem: drop(content + position)
    self.tok_emb = nn.Embedding(vocab_size, n_embd)
    self.pos_emb = nn.Parameter(torch.zeros(1, block_size, n_embd))
    self.drop = nn.Dropout(embd_pdrop)

  @typecheck()
  def forward(self, idx):
    b, t = idx.size()

    # forward the GPT model
    token_embeddings = self.tok_emb(idx) # each index maps to a (learnable) vector
    position_embeddings = self.pos_emb[:, :t, :] # each position maps to a (learnable) vector
    x = self.drop(token_embeddings + position_embeddings)
    return x

  @property
  def input_types(self):
    return {
        'idx': NeuralType(('B', 'T'), Index())
    }

  @property
  def output_types(self):
    return {
        'embeddings': NeuralType(('B', 'T', 'C'), EmbeddedTextType())
    }

### Refactoring the Encoder

Next, let's refactor the GPT Encoder - which is implemented as a multi layer Transformer (Decoder) network.

------
It can be noted that we refer to the GPT "Encoder" module - but it is constructed by using Transformer "Decoder" blocks.

***When we discuss Neural Modules - we are discussing an abstract module with a certain input neural type and a certain output neural type.***

For us, the GPT "Encoder" neural module will accept any  implementation, whose

- input neural type is `NeuralType(('B', 'T', 'C'), EmbeddedTextType())`

- output type is `NeuralType(('B', 'T', 'C'), EncodedRepresentation())`

-----
One concrete implementation of such a GPT "Encoder" neural module is a Deep Transformer "Decoder" network.

In [33]:
class GPTTransformerEncoder(NeuralModule):
  def __init__(self, n_embd: int, block_size: int, n_head: int, n_layer: int, attn_pdrop: float = 0.0, resid_pdrop: float = 0.0):
    super().__init__()

    self.blocks = nn.Sequential(*[Block(n_embd, block_size, n_head, attn_pdrop, resid_pdrop)
                                  for _ in range(n_layer)])

  @typecheck()
  def forward(self, embed):
    return self.blocks(embed)

  @property
  def input_types(self):
    return {
        'embed': NeuralType(('B', 'T', 'C'), EmbeddedTextType())
    }

  @property
  def output_types(self):
    return {
        'encoding': NeuralType(('B', 'T', 'C'), CausalSelfAttentionType())
    }

### Refactoring the Decoder

Finally, let's refactor the Decoder - the small one-layer feed-forward network to decode the answer.

-------

Note an interesting detail - The `input_types` of the Decoder accepts the generic `EncoderRepresentation()`, where as the `neural_type` of the `GPTTransformerEncoder` has the `output_type` of `CausalSelfAttentionType`.

This is semantically *not* a mismatch! As you can see above in the inheritance chart, we declare `EncodedRepresentation` -> `AttentionType` -> `SelfAttentionType` -> `CausalSelfAttentionType`.

Such an inheritance hierarchy for the `element_type` allows future encoders (which also have a neural output type of at least `EncodedRepresentation`) to be swapped in place of the current GPT Causal Self Attention Encoder while keeping the rest of the NeMo model working just fine!

In [34]:
class GPTDecoder(NeuralModule):
  def __init__(self, n_embd: int, vocab_size: int):
    super().__init__()
    self.ln_f = nn.LayerNorm(n_embd)
    self.head = nn.Linear(n_embd, vocab_size, bias=False) # no need for extra bias due to one in ln_f

  @typecheck()
  def forward(self, encoding):
    x = self.ln_f(encoding)
    logits = self.head(x)
    return logits

  @property
  def input_types(self):
    return {
        'encoding': NeuralType(('B', 'T', 'C'), EncodedRepresentation())
    }

  @property
  def output_types(self):
    return {
        'logits': NeuralType(('B', 'T', 'C'), LogitsType())
    }


### Refactoring the NeMo GPT Model

Now that we have 3 NeuralModules for the embedding, the encoder, and the decoder, let's refactor the NeMo model to take advantage of this refactor!

This time, we inherit from `ModelPT` instead of the general `LightningModule`.

In [35]:
class AbstractNeMoGPT(ModelPT):
  def __init__(self, cfg: OmegaConf, trainer: ptl.Trainer = None):
      super().__init__(cfg=cfg, trainer=trainer)

      # input embedding stem: drop(content + position)
      self.embedding = self.from_config_dict(self.cfg.embedding)
      # deep transformer: just a sequence of transformer blocks
      self.encoder = self.from_config_dict(self.cfg.encoder)
      # decoder: at the end one more layernorm and decode the answers
      self.decoder = self.from_config_dict(self.cfg.decoder)

      self.block_size = self.cfg.embedding.block_size
      self.apply(self._init_weights)

      print("number of parameters: %e" % self.num_weights)

  @typecheck()
  def forward(self, idx):
      b, t = idx.size()
      assert t <= self.block_size, "Cannot forward, model block size is exhausted."

      # forward the GPT model
      # Remember: Only kwargs are allowed !
      e = self.embedding(idx=idx)
      x = self.encoder(embed=e)
      logits = self.decoder(encoding=x)

      return logits

  def get_block_size(self):
      return self.block_size

  def _init_weights(self, module):
      """
      Vanilla model initialization:
      - all MatMul weights \in N(0, 0.02) and biases to zero
      - all LayerNorm post-normalization scaling set to identity, so weight=1, bias=0
      """
      if isinstance(module, (nn.Linear, nn.Embedding)):
          module.weight.data.normal_(mean=0.0, std=0.02)
          if isinstance(module, nn.Linear) and module.bias is not None:
              module.bias.data.zero_()
      elif isinstance(module, nn.LayerNorm):
          module.bias.data.zero_()
          module.weight.data.fill_(1.0)

  @property
  def input_types(self):
    return {
        'idx': NeuralType(('B', 'T'), Index())
    }

  @property
  def output_types(self):
    return {
        'logits': NeuralType(('B', 'T', 'C'), LogitsType())
    }

## Creating a config for a Model

At first glance, not much changed compared to the PyTorch Lightning implementation above. Other than the constructor, which now accepts a config, nothing changed at all!

NeMo operates on the concept of a NeMo Model being accompanied by a corresponding config dict (instantiated as an OmegaConf object). This enables us to prototype the model by utilizing Hydra rapidly. This includes various other benefits - such as hyperparameter optimization and serialization/deserialization of NeMo models.

Let's look at how actually to construct such config objects!

In [36]:
# model definition args (required)
# ================================
# vocab_size: int # size of the vocabulary (number of possible tokens)
# block_size: int # length of the model's context window in time
# n_layer: int # depth of the model; number of Transformer blocks in sequence
# n_embd: int # the "width" of the model, number of channels in each Transformer
# n_head: int # number of heads in each multi-head attention inside each Transformer block

# model definition args (optional)
# ================================
# embd_pdrop: float = 0.1, # \in [0,1]: amount of dropout on input embeddings
# resid_pdrop: float = 0.1, # \in [0,1]: amount of dropout in each residual connection
# attn_pdrop: float = 0.1, # \in [0,1]: amount of dropout on the attention matrix

------
As we look at the required parameters above, we need a way to tell OmegaConf that these values are currently not set, but the user should set them before we use them.

OmegaConf supports such behavior using the `MISSING` value. A similar effect can be achieved in YAML configs by using `???` as a placeholder.

In [37]:
from omegaconf import MISSING

In [38]:
# Let's create a utility for building the class path
def get_class_path(cls):
  return f'{cls.__module__}.{cls.__name__}'

### Structure of a Model config

Let's first create a config for the common components of the model level config -

In [39]:
common_config = OmegaConf.create({
    'vocab_size': MISSING,
    'block_size': MISSING,
    'n_layer': MISSING,
    'n_embd': MISSING,
    'n_head': MISSING,
})

-----
The model config right now is still being built - it needs to contain a lot more details!

A complete Model Config should have the sub-configs of all of its top-level modules as well. This means the configs of the `embedding`, `encoder`, and the `decoder`.


### Structure of sub-module config

For top-level models, we generally don't change the actual module very often, and instead, primarily change the hyperparameters of that model.

So we will make use of `Hydra`'s Class instantiation method - which can easily be accessed via the class method `ModelPT.from_config_dict()`.

Let's take a few examples below -

In [40]:
embedding_config = OmegaConf.create({
    '_target_': get_class_path(GPTEmbedding),
    'vocab_size': '${model.vocab_size}',
    'n_embd': '${model.n_embd}',
    'block_size': '${model.block_size}',
    'embd_pdrop': 0.1
})

encoder_config = OmegaConf.create({
    '_target_': get_class_path(GPTTransformerEncoder),
    'n_embd': '${model.n_embd}',
    'block_size': '${model.block_size}',
    'n_head': '${model.n_head}',
    'n_layer': '${model.n_layer}',
    'attn_pdrop': 0.1,
    'resid_pdrop': 0.1
})

decoder_config = OmegaConf.create({
    '_target_': get_class_path(GPTDecoder),
    # n_embd: int, vocab_size: int
    'n_embd': '${model.n_embd}',
    'vocab_size': '${model.vocab_size}'
})

##### What is `_target_`?
--------

In the above config, we see a `_target_` in the config. `_target_` is usually a full classpath to the actual class in the python package/user local directory. It is required for Hydra to locate and instantiate the model from its path correctly.

So why do we want to set a classpath?

In general, when developing models, we don't often change the encoder or the decoder, but we do change the hyperparameters of the encoder and decoder.

This notation helps us keep the Model level declaration of the forward step neat and precise. It also logically helps us demark which parts of the model can be easily replaced - in the future, we can easily replace the encoder with some other type of self-attention block or the decoder with an RNN or 1D-CNN neural module (as long as they have the same Neural Type definition as the current blocks).


##### What is the `${}` syntax?
-------

OmegaConf, and by extension, Hydra, supports Variable Interpolation. As you can see in the `__init__` of embedding, encoder, and decoder neural modules, they often share many parameters between each other.

It would become tedious and error-prone to set each of these constructors' values separately in each of the embedding, encoder, and decoder configs.

So instead, we define standard keys inside of the `model` level config and then interpolate these values inside of the respective configs!

### Attaching the model and module-level configs

So now, we have a Model level and per-module level configs for the core components. Sub-module configs generally fall under the "model" namespace, but you have the flexibility to define the structure as you require.

Let's attach them!


In [41]:
model_config = OmegaConf.create({
    'model': common_config
})

# Then let's attach the sub-module configs
model_config.model.embedding = embedding_config
model_config.model.encoder = encoder_config
model_config.model.decoder = decoder_config

-----
Let's print this config!

In [42]:
print(OmegaConf.to_yaml(model_config))

model:
  vocab_size: ???
  block_size: ???
  n_layer: ???
  n_embd: ???
  n_head: ???
  embedding:
    _target_: __main__.GPTEmbedding
    vocab_size: ${model.vocab_size}
    n_embd: ${model.n_embd}
    block_size: ${model.block_size}
    embd_pdrop: 0.1
  encoder:
    _target_: __main__.GPTTransformerEncoder
    n_embd: ${model.n_embd}
    block_size: ${model.block_size}
    n_head: ${model.n_head}
    n_layer: ${model.n_layer}
    attn_pdrop: 0.1
    resid_pdrop: 0.1
  decoder:
    _target_: __main__.GPTDecoder
    n_embd: ${model.n_embd}
    vocab_size: ${model.vocab_size}



-----
Wait, why did OmegaConf not fill in the value of the variable interpolation for the configs yet?

This is because OmegaConf takes a deferred approach to variable interpolation. First, we fill in temporary values of the required fields (those marked by `???`). Then, to force resolution ahead of time, we can use the following snippet -

In [43]:
import copy

In [44]:
temp_config = copy.deepcopy(model_config)
temp_config.model.vocab_size = 10
temp_config.model.block_size = 4
temp_config.model.n_layer = 1
temp_config.model.n_embd = 32
temp_config.model.n_head = 4

temp_config = OmegaConf.create(OmegaConf.to_container(temp_config, resolve=True))
print(OmegaConf.to_yaml(temp_config))

model:
  vocab_size: 10
  block_size: 4
  n_layer: 1
  n_embd: 32
  n_head: 4
  embedding:
    _target_: __main__.GPTEmbedding
    vocab_size: 10
    n_embd: 32
    block_size: 4
    embd_pdrop: 0.1
  encoder:
    _target_: __main__.GPTTransformerEncoder
    n_embd: 32
    block_size: 4
    n_head: 4
    n_layer: 1
    attn_pdrop: 0.1
    resid_pdrop: 0.1
  decoder:
    _target_: __main__.GPTDecoder
    n_embd: 32
    vocab_size: 10



-----
Now that we have a config, let's try to create an object of the NeMo Model !

In [45]:
# Let's work on a copy of the model config and update it before we send it into the Model.
cfg = copy.deepcopy(model_config)

In [46]:
# Let's set the values of the config (for some plausible small model)
cfg.model.vocab_size = 100
cfg.model.block_size = 128
cfg.model.n_layer = 1
cfg.model.n_embd = 32
cfg.model.n_head = 4

In [47]:
print(OmegaConf.to_yaml(cfg))

model:
  vocab_size: 100
  block_size: 128
  n_layer: 1
  n_embd: 32
  n_head: 4
  embedding:
    _target_: __main__.GPTEmbedding
    vocab_size: ${model.vocab_size}
    n_embd: ${model.n_embd}
    block_size: ${model.block_size}
    embd_pdrop: 0.1
  encoder:
    _target_: __main__.GPTTransformerEncoder
    n_embd: ${model.n_embd}
    block_size: ${model.block_size}
    n_head: ${model.n_head}
    n_layer: ${model.n_layer}
    attn_pdrop: 0.1
    resid_pdrop: 0.1
  decoder:
    _target_: __main__.GPTDecoder
    n_embd: ${model.n_embd}
    vocab_size: ${model.vocab_size}



In [48]:
# Try to create a model with this config [ERROR CELL]
m = AbstractNeMoGPT(cfg.model)

TypeError: Can't instantiate abstract class AbstractNeMoGPT with abstract methods list_available_models, setup_training_data, setup_validation_data

-----

You will note that we added the `Abstract` tag for a reason to this NeMo Model and that when we try to instantiate it - it raises an error that we need to implement specific methods.

1) `setup_training_data` & `setup_validation_data` - All NeMo models should implement two data loaders - the training data loader and the validation data loader. Optionally, they can go one step further and also implement the `setup_test_data` method to add support for evaluating the Model on its own.

Why do we enforce this? NeMo Models are meant to be a unified, cohesive object containing the details about the neural network underlying that Model and the data loaders to train, validate, and optionally test those models.

In doing so, once the Model is created/deserialized, it would take just a few more steps to train the Model from scratch / fine-tune/evaluate the Model on any data that the user provides, as long as this user-provided dataset is in a format supported by the Dataset / DataLoader that is used by this Model!

2) `list_available_models` - This is a utility method to provide a list of pre-trained NeMo models to the user from the cloud.

Typically, NeMo models can be easily packaged into a tar file (which we call a .nemo file in the earlier primer notebook). These tar files contain the model config + the pre-trained checkpoint weights of the Model, and can easily be downloaded from some cloud service.

For this notebook, we will not be implementing this method.

--------
Finally, let's create a concrete implementation of the above NeMo Model!

In [49]:
from nemo.core.classes.common import PretrainedModelInfo

In [50]:
class BasicNeMoGPT(AbstractNeMoGPT):

  @classmethod
  def list_available_models(cls) -> PretrainedModelInfo:
    return None

  def setup_training_data(self, train_data_config: OmegaConf):
    self._train_dl = None

  def setup_validation_data(self, val_data_config: OmegaConf):
    self._validation_dl = None

  def setup_test_data(self, test_data_config: OmegaConf):
    self._test_dl = None

------
Now let's try to create an object of the `BasicNeMoGPT` model

In [51]:
m = BasicNeMoGPT(cfg.model)

number of parameters: 2.326400e+04


## Setting up train-val-test steps

The above `BasicNeMoGPT` Model is a basic PyTorch Lightning Module, with some added functionality -

1) Neural Type checks support - as defined in the Model as well as the internal modules.

2) Save and restore of the Model (in the trivial case) to a tarfile.

But as the Model is right now, it crucially does not support PyTorch Lightning's `Trainer`. As such, while this Model can be called manually, it cannot be easily trained or evaluated by using the PyTorch Lightning framework.

------

Let's begin adding support for this then -

In [52]:
class BasicNeMoGPTWithSteps(BasicNeMoGPT):

    def step_(self, split, batch, batch_idx=None):
        idx, targets = batch
        logits = self(idx=idx)
        loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        key = 'loss' if split == 'train' else f"{split}_loss"
        self.log(key, loss)
        return {key: loss}

    def training_step(self, *args, **kwargs):
        return self.step_('train', *args, **kwargs)

    def validation_step(self, *args, **kwargs):
        return self.step_('val', *args, **kwargs)

    def test_step(self, *args, **kwargs):
        return self.step_('test', *args, **kwargs)

    # This is useful for multiple validation data loader setup
    def multi_validation_epoch_end(self, outputs, dataloader_idx: int = 0):
        val_loss_mean = torch.stack([x['val_loss'] for x in outputs]).mean()
        return {'val_loss': val_loss_mean}

    # This is useful for multiple test data loader setup
    def multi_test_epoch_end(self, outputs, dataloader_idx: int = 0):
        test_loss_mean = torch.stack([x['test_loss'] for x in outputs]).mean()
        return {'test_loss': test_loss_mean}

In [ ]:
m = BasicNeMoGPTWithSteps(cfg=cfg.model)

### Setup for Multi Validation and Multi Test data loaders

As discussed in the NeMo Primer, NeMo has in-built support for multiple data loaders for validation and test steps. Therefore, as an example of how easy it is to add such support, we include the `multi_validation_epoch_end` and `multi_test_epoch_end` overrides.

It is also practically essential to collate results from more than one distributed GPUs, and then aggregate results properly at the end of the epoch. NeMo strictly enforces the correct collation of results, even if you will work on only one device! Future-proofing is baked into the model design for this case!

Therefore NeMo provides the above two generic methods to support aggregation and simultaneously support multiple datasets!

**Please note, you can prepend your already existing `on_validation_epoch_end` and `on_test_epoch_end` implementations with the `multi_` in the name, and that alone is sufficient to enable multi-dataset and multi-GPU support!**

------
**Note: To disable multi-dataset support, simply override `on_validation_epoch_end` and `on_test_epoch_end` instead of `multi_validation_epoch_end` and `multi_test_epoch_end`!**

## Setting up the optimizer / scheduler

We are relatively close to reaching feature parity with the MinGPT Model! But we are missing a crucial piece - the optimizer.

All NeMo Model's come with a default implementation of `setup_optimization()`, which will parse the provided model config to obtain the `optim` and `sched` sub-configs, and automatically configure the optimizer and scheduler.

If training GPT was as simple as plugging in an Adam optimizer over all the parameters with a cosine weight decay schedule, we could do that from the config alone.

-------

But GPT is not such a trivial model - more specifically, it requires weight decay to be applied to the weight matrices but not to the biases, the embedding matrix, or the LayerNorm layers.

We can drop the support that Nemo provides for such special cases and instead utilize the PyTorch Lightning method `configure_optimizers` to perform the same task.

-------

Note, for NeMo Models; the `configure_optimizers` is implemented as a trivial call to `setup_optimization()` followed by returning the generated optimizer and scheduler! So we can override the `configure_optimizer` method and manage the optimizer creation manually!

NeMo's goal is to provide usable defaults for the general case and simply back off to either PyTorch Lightning or PyTorch nn.Module itself in cases when the additional flexibility becomes necessary!

In [53]:
class BasicNeMoGPTWithOptim(BasicNeMoGPTWithSteps):

     def configure_optimizers(self):
        """
        This long function is unfortunately doing something very simple and is being very defensive:
        We are separating out all parameters of the model into two buckets: those that will experience
        weight decay for regularization and those that won't (biases, and layernorm/embedding weights).
        We are then returning the PyTorch optimizer object.
        """

        # separate out all parameters to those that will and won't experience weight decay
        decay = set()
        no_decay = set()
        whitelist_weight_modules = (torch.nn.Linear, )
        blacklist_weight_modules = (torch.nn.LayerNorm, torch.nn.Embedding)
        for mn, m in self.named_modules():
            for pn, p in m.named_parameters():
                fpn = '%s.%s' % (mn, pn) if mn else pn # full param name

                if pn.endswith('bias'):
                    # all biases will not be decayed
                    no_decay.add(fpn)
                elif pn.endswith('weight') and isinstance(m, whitelist_weight_modules):
                    # weights of whitelist modules will be weight decayed
                    decay.add(fpn)
                elif pn.endswith('weight') and isinstance(m, blacklist_weight_modules):
                    # weights of blacklist modules will NOT be weight decayed
                    no_decay.add(fpn)

        # special case the position embedding parameter in the root GPT module as not decayed
        no_decay.add('embedding.pos_emb')

        # validate that we considered every parameter
        param_dict = {pn: p for pn, p in self.named_parameters()}
        inter_params = decay & no_decay
        union_params = decay | no_decay
        assert len(inter_params) == 0, "parameters %s made it into both decay/no_decay sets!" % (str(inter_params), )
        assert len(param_dict.keys() - union_params) == 0, "parameters %s were not separated into either decay/no_decay set!" \
                                                    % (str(param_dict.keys() - union_params), )

        # create the pytorch optimizer object
        optim_groups = [
            {"params": [param_dict[pn] for pn in sorted(list(decay))], "weight_decay": self.cfg.optim.weight_decay},
            {"params": [param_dict[pn] for pn in sorted(list(no_decay))], "weight_decay": 0.0},
        ]
        optimizer = torch.optim.AdamW(optim_groups, lr=self.cfg.optim.lr, betas=self.cfg.optim.betas)
        return optimizer


In [54]:
m = BasicNeMoGPTWithOptim(cfg=cfg.model)

number of parameters: 2.326400e+04


-----
Now let's setup the config for the optimizer !

In [55]:
OmegaConf.set_struct(cfg.model, False)

optim_config = OmegaConf.create({
    'lr': 3e-4,
    'weight_decay': 0.1,
    'betas': [0.9, 0.95]
})

cfg.model.optim = optim_config

OmegaConf.set_struct(cfg.model, True)

## Setting up the dataset / data loaders

So we were able almost entirely to replicate the MinGPT implementation.

Remember, NeMo models should contain all of the logic to load the Dataset and DataLoader for at least the train and validation step.

We temporarily provided empty implementations to get around it till now, but let's fill that in now!

-------

**Note for datasets**: Below, we will show an example using a very small dataset called `tiny_shakespeare`, found at the original [char-rnn repository](https://github.com/karpathy/char-rnn), but practically you could use any text corpus. The one suggested in minGPT is available at http://mattmahoney.net/dc/textdata.html

### Creating the Dataset

NeMo has Neural Type checking support, even for Datasets! It's just a minor change of the import in most cases and one difference in how we handle `collate_fn`.

We could paste the dataset info from minGPT, and you'd only need to make 2 changes!

-----
In this example, we will be writing a thin subclass over the datasets provided by `nlp` from HuggingFace!

In [56]:
from nemo.core import Dataset
from torch.utils import data
from torch.utils.data.dataloader import DataLoader

In [57]:
class TinyShakespeareDataset(Dataset):

  def __init__(self, data_path, block_size, crop=None, override_vocab=None):

      # load the data and crop it appropriately
      with open(data_path, 'r') as f:
          if crop is None:
              data = f.read()
          else:
              f.seek(crop[0])
              data = f.read(crop[1])

      # build a vocabulary from data or inherit it
      vocab = sorted(list(set(data))) if override_vocab is None else override_vocab

      # Add UNK
      special_tokens = ['<PAD>', '<UNK>']  # We use just <UNK> and <PAD> in the call, but can add others.
      if not override_vocab:
        vocab = [*special_tokens, *vocab]  # Update train vocab with special tokens

      data_size, vocab_size = len(data), len(vocab)
      print('data of crop %s has %d characters, vocab of size %d.' % (str(crop), data_size, vocab_size))
      print('Num samples in dataset : %d' % (data_size // block_size))

      self.stoi = { ch:i for i,ch in enumerate(vocab) }
      self.itos = { i:ch for i,ch in enumerate(vocab) }
      self.block_size = block_size
      self.vocab_size = vocab_size
      self.data = data
      self.vocab = vocab
      self.special_tokens = special_tokens

  def __len__(self):
      return len(self.data) // self.block_size

  def __getitem__(self, idx):
      # attempt to fetch a chunk of (block_size + 1) items, but (block_size) will work too
      chunk = self.data[idx*self.block_size : min(len(self.data), (idx+1)*self.block_size + 1)]
      # map the string into a sequence of integers
      ixes = [self.stoi[s] if s in self.stoi else self.stoi['<UNK>'] for s in chunk ]
      # if stars align (last idx and len(self.data) % self.block_size == 0), pad with <PAD>
      if len(ixes) < self.block_size + 1:
          assert len(ixes) == self.block_size # i believe this is the only way this could happen, make sure
          ixes.append(self.stoi['<PAD>'])
      dix = torch.tensor(ixes, dtype=torch.long)
      return dix[:-1], dix[1:]

  @property
  def output_types(self):
    return {
        'input': NeuralType(('B', 'T'), Index()),
        'target': NeuralType(('B', 'T'), LabelsType())
    }

------
We didn't have to change anything until here. How then is type-checking done?

NeMo does type-checking inside of the collate function implementation itself! In this case, it is not necessary to override the `collate_fn` inside the Dataset, but if we did need to override it, **NeMo requires that the private method `_collate_fn` be overridden instead**.

We can then use data loaders with minor modifications!

**Also, there is no need to implement the `input_types` for Dataset, as they are the ones generating the input for the model!**

-----
Let's prepare the dataset that we are going to use - Tiny Shakespeare from the following codebase [char-rnn](https://github.com/karpathy/char-rnn).

In [58]:
import os

In [59]:
if not os.path.exists('tiny-shakespeare.txt'):
  !wget https://raw.githubusercontent.com/jcjohnson/torch-rnn/master/data/tiny-shakespeare.txt

--2025-03-27 10:57:48--  https://raw.githubusercontent.com/jcjohnson/torch-rnn/master/data/tiny-shakespeare.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘tiny-shakespeare.txt’

tiny-shakespeare.tx 100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2025-03-27 10:57:49 (21.8 MB/s) - ‘tiny-shakespeare.txt’ saved [1115394/1115394]



In [60]:
!head -n 5 tiny-shakespeare.txt

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


In [61]:
train_dataset = TinyShakespeareDataset('tiny-shakespeare.txt', cfg.model.block_size, crop=(0,         int(1e6)))
val_dataset   = TinyShakespeareDataset('tiny-shakespeare.txt', cfg.model.block_size, crop=(int(1e6), int(50e3)), override_vocab=train_dataset.vocab)
test_dataset  = TinyShakespeareDataset('tiny-shakespeare.txt', cfg.model.block_size, crop=(int(1.05e6), int(100e3)), override_vocab=train_dataset.vocab)

data of crop (0, 1000000) has 1000000 characters, vocab of size 67.
Num samples in dataset : 7812
data of crop (1000000, 50000) has 50000 characters, vocab of size 67.
Num samples in dataset : 390
data of crop (1050000, 100000) has 65394 characters, vocab of size 67.
Num samples in dataset : 510


### Setting up dataset/data loader support in the Model

So we now know our data loader works. Let's integrate it as part of the Model itself!

To do this, we use the three special attributes of the NeMo Model - `self._train_dl`, `self._validation_dl` and `self._test_dl`. Once you construct your DataLoader, place your data loader to these three variables.

For multi-data loader support, the same applies! NeMo will automatically handle the management of multiple data loaders for you!

In [62]:
class NeMoGPT(BasicNeMoGPTWithOptim):

  def _setup_data_loader(self, cfg):
    if self.vocab is None:
      override_vocab = None
    else:
      override_vocab = self.vocab

    dataset = TinyShakespeareDataset(
        data_path=cfg.data_path,
        block_size=cfg.block_size,
        crop=tuple(cfg.crop) if 'crop' in cfg else None,
        override_vocab=override_vocab
    )

    if self.vocab is None:
      self.vocab = dataset.vocab

    return DataLoader(
        dataset=dataset,
        batch_size=cfg.batch_size,
        shuffle=cfg.shuffle,
        collate_fn=dataset.collate_fn,  # <-- this is necessary for type checking
        pin_memory=cfg.pin_memory if 'pin_memory' in cfg else False,
        num_workers=cfg.num_workers if 'num_workers' in cfg else 0
    )

  def setup_training_data(self, train_data_config: OmegaConf):
    self.vocab = None
    self._train_dl = self._setup_data_loader(train_data_config)

  def setup_validation_data(self, val_data_config: OmegaConf):
    self._validation_dl = self._setup_data_loader(val_data_config)

  def setup_test_data(self, test_data_config: OmegaConf):
    self._test_dl = self._setup_data_loader(test_data_config)


### Creating the dataset / dataloader config

The final step to setup this model is to add the `train_ds`, `validation_ds` and `test_ds` configs inside the model config!

In [63]:
OmegaConf.set_struct(cfg.model, False)

# Set the data path and update vocabular size
cfg.model.data_path = 'tiny-shakespeare.txt'
cfg.model.vocab_size = train_dataset.vocab_size

OmegaConf.set_struct(cfg.model, True)

In [64]:
train_ds = OmegaConf.create({
    'data_path': '${model.data_path}',
    'block_size': '${model.block_size}',
    'crop': [0, int(1e6)],
    'batch_size': 64,
    'shuffle': True,
})

validation_ds = OmegaConf.create({
    'data_path': '${model.data_path}',
    'block_size': '${model.block_size}',
    'crop': [int(1e6), int(50e3)],
    'batch_size': 4,
    'shuffle': False,
})

test_ds = OmegaConf.create({
    'data_path': '${model.data_path}',
    'block_size': '${model.block_size}',
    'crop': [int(1.05e6), int(100e3)],
    'batch_size': 4,
    'shuffle': False,
})

In [65]:
# Attach to the model config
OmegaConf.set_struct(cfg.model, False)

cfg.model.train_ds = train_ds
cfg.model.validation_ds = validation_ds
cfg.model.test_ds = test_ds

OmegaConf.set_struct(cfg.model, True)

In [66]:
# Let's see the config now !
print(OmegaConf.to_yaml(cfg))

model:
  vocab_size: 67
  block_size: 128
  n_layer: 1
  n_embd: 32
  n_head: 4
  embedding:
    _target_: __main__.GPTEmbedding
    vocab_size: ${model.vocab_size}
    n_embd: ${model.n_embd}
    block_size: ${model.block_size}
    embd_pdrop: 0.1
  encoder:
    _target_: __main__.GPTTransformerEncoder
    n_embd: ${model.n_embd}
    block_size: ${model.block_size}
    n_head: ${model.n_head}
    n_layer: ${model.n_layer}
    attn_pdrop: 0.1
    resid_pdrop: 0.1
  decoder:
    _target_: __main__.GPTDecoder
    n_embd: ${model.n_embd}
    vocab_size: ${model.vocab_size}
  optim:
    lr: 0.0003
    weight_decay: 0.1
    betas:
    - 0.9
    - 0.95
  data_path: tiny-shakespeare.txt
  train_ds:
    data_path: ${model.data_path}
    block_size: ${model.block_size}
    crop:
    - 0
    - 1000000
    batch_size: 64
    shuffle: true
  validation_ds:
    data_path: ${model.data_path}
    block_size: ${model.block_size}
    crop:
    - 1000000
    - 50000
    batch_size: 4
    shuffle: false


In [67]:
# Let's try creating a model now !
model = NeMoGPT(cfg=cfg.model)

data of crop (0, 1000000) has 1000000 characters, vocab of size 67.
Num samples in dataset : 7812
data of crop (1000000, 50000) has 50000 characters, vocab of size 67.
Num samples in dataset : 390
data of crop (1050000, 100000) has 65394 characters, vocab of size 67.
Num samples in dataset : 510
number of parameters: 2.115200e+04


-----
All the data loaders load properly ! Yay!

# Evaluate the model - end to end!

Now that the data loaders have been set up, all that's left is to train and test the model! We have most of the components required by this model - the train, val and test data loaders, the optimizer, and the type-checked forward step to perform the train-validation-test steps!

But training a GPT model from scratch is not the goal of this primer, so instead, let's do a sanity check by merely testing the model for a few steps using random initial weights.

The above will ensure that -

1) Our data loaders work as intended

2) The type checking system assures us that our Neural Modules are performing their forward step correctly.

3) The loss is calculated, and therefore the model runs end to end, ultimately supporting PyTorch Lightning.

In [68]:
if torch.cuda.is_available():
  accelerator = 'gpu'
else:
  accelerator = 'cpu'

trainer = ptl.Trainer(devices=1, accelerator=accelerator, limit_test_batches=1.0)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: `Trainer(limit_test_batches=1.0)` was configured so 100% of the batches will be used..
INFO:lightning.pytorch.utilities.rank_zero:`Trainer(limit_test_batches=1.0)` was configured so 100% of the batches will be used..


In [72]:
trainer.test(model)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │     4.228049278259277     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 4.228049278259277}]

# Saving and restoring models

NeMo internally keeps track of the model configuration, as well as the model checkpoints and parameters.

As long as your NeMo follows the above general guidelines, you can call the `save_to` and `restore_from` methods to save and restore your models!

In [73]:
model.save_to('gpt_model.nemo')

In [74]:
!ls -d -- *.nemo

gpt_model.nemo


In [75]:
temp_model = NeMoGPT.restore_from('gpt_model.nemo')

[NeMo W 2025-03-27 11:04:42 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    data_path: tiny-shakespeare.txt
    block_size: 128
    crop:
    - 0
    - 1000000
    batch_size: 64
    shuffle: true
    
[NeMo W 2025-03-27 11:04:42 nemo_logging:405] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    data_path: tiny-shakespeare.txt
    block_size: 128
    crop:
    - 1000000
    - 50000
    batch_size: 4
    shuffle: false
    
[NeMo W 2025-03-27 11:04:42 nemo_logging:405] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple_test_data() method and provide a valid configuration file to setup the test data loader(s).
    Test conf

number of parameters: 2.115200e+04
[NeMo I 2025-03-27 11:04:42 nemo_logging:393] Model NeMoGPT was successfully restored from /content/gpt_model.nemo.


In [76]:
# [ERROR CELL]
temp_model.setup_test_data(temp_model.cfg.test_ds)

AttributeError: 'NeMoGPT' object has no attribute 'vocab'

-----

Hmm, it seems it wasn't so easy in this case. Non-trivial models have non-trivial issues!

Remember, our NeMoGPT model sets its self.vocab inside the `setup_train_data` step. But that depends on the vocabulary generated by the train set... which is **not** restored during model restoration (unless you call `setup_train_data` explicitly!).

We can quickly resolve this issue by constructing an external data file to enable save and restore support, and NeMo supports that too! We will use the `register_artifact` API in NeMo to support external files being attached to the .nemo checkpoint.

In [77]:
class NeMoGPTv2(NeMoGPT):

  def setup_training_data(self, train_data_config: OmegaConf):
    self.vocab = None
    self._train_dl = self._setup_data_loader(train_data_config)

    # Save the vocab into a text file for now
    with open('vocab.txt', 'w') as f:
      for token in self.vocab:
        f.write(f"{token}<SEP>")

    # This is going to register the file into .nemo!
    # When you later use .save_to(), it will copy this file into the tar file.
    self.register_artifact('vocab_file', 'vocab.txt')

  def setup_validation_data(self, val_data_config: OmegaConf):
    # This is going to try to find the same file, and if it fails,
    # it will use the copy in .nemo
    vocab_file = self.register_artifact('vocab_file', 'vocab.txt')

    with open(vocab_file, 'r') as f:
      vocab = []
      vocab = f.read().split('<SEP>')[:-1]  # the -1 here is for the dangling <SEP> token in the file
      self.vocab = vocab

    self._validation_dl = self._setup_data_loader(val_data_config)

  def setup_test_data(self, test_data_config: OmegaConf):
    # This is going to try to find the same file, and if it fails,
    # it will use the copy in .nemo
    vocab_file = self.register_artifact('vocab_file', 'vocab.txt')

    with open(vocab_file, 'r') as f:
      vocab = []
      vocab = f.read().split('<SEP>')[:-1]  # the -1 here is for the dangling <SEP> token in the file
      self.vocab = vocab

    self._test_dl = self._setup_data_loader(test_data_config)


In [78]:
# Let's try creating a model now !
model = NeMoGPTv2(cfg=cfg.model)

[NeMo W 2025-03-27 11:05:12 nemo_logging:405] You tried to register an artifact under config key=vocab_file but an artifact for it has already been registered.
[NeMo W 2025-03-27 11:05:12 nemo_logging:405] You tried to register an artifact under config key=vocab_file but an artifact for it has already been registered.


data of crop (0, 1000000) has 1000000 characters, vocab of size 67.
Num samples in dataset : 7812
data of crop (1000000, 50000) has 50000 characters, vocab of size 67.
Num samples in dataset : 390
data of crop (1050000, 100000) has 65394 characters, vocab of size 67.
Num samples in dataset : 510
number of parameters: 2.115200e+04


In [79]:
# Now let's try to save and restore !
model.save_to('gpt_model.nemo')

In [80]:
temp_model = NeMoGPTv2.restore_from('gpt_model.nemo')

[NeMo W 2025-03-27 11:05:16 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    data_path: tiny-shakespeare.txt
    block_size: 128
    crop:
    - 0
    - 1000000
    batch_size: 64
    shuffle: true
    
[NeMo W 2025-03-27 11:05:16 nemo_logging:405] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    data_path: tiny-shakespeare.txt
    block_size: 128
    crop:
    - 1000000
    - 50000
    batch_size: 4
    shuffle: false
    
[NeMo W 2025-03-27 11:05:16 nemo_logging:405] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple_test_data() method and provide a valid configuration file to setup the test data loader(s).
    Test conf

number of parameters: 2.115200e+04
[NeMo I 2025-03-27 11:05:16 nemo_logging:393] Model NeMoGPTv2 was successfully restored from /content/gpt_model.nemo.


In [81]:
temp_model.setup_multiple_test_data(temp_model.cfg.test_ds)

data of crop (1050000, 100000) has 65394 characters, vocab of size 67.
Num samples in dataset : 510


In [82]:
if torch.cuda.is_available():
  accelerator = 'gpu'
else:
  accelerator = 'cpu'

trainer = ptl.Trainer(devices=1, accelerator=accelerator, limit_test_batches =1.0)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: `Trainer(limit_test_batches=1.0)` was configured so 100% of the batches will be used..
INFO:lightning.pytorch.utilities.rank_zero:`Trainer(limit_test_batches=1.0)` was configured so 100% of the batches will be used..


In [83]:
trainer.test(model)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │     4.205028533935547     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 4.205028533935547}]

------
There we go ! Now our models can be serialized and de-serialized without any issue, even with an external vocab file !